In [74]:
import pandas as pd
import string
import nltk
import re
import pickle
stopwords = nltk.corpus.stopwords.words('english')

ps = nltk.PorterStemmer()

df_train = pd.read_csv('atis_intents_train.csv',header=None)

df_train.columns = ['label','text']

#Create function to remove punctuation, tokenize, remove stopwords, and stem

In [75]:
def clean_text(text):
    text = ''.join([word.lower() for word in text if text not in string.punctuation])
    tokens = re.split('\W+', text)
    text = ' '.join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

In [76]:
df_train['cleaned_text'] = df_train['text'].apply(lambda x : clean_text(x))

In [77]:
df_train.head()

,label,text,cleaned_text
0,atis_flight,i want to fly from boston at 838 am and arriv...,want fli boston 838 arriv denver 1110 morn
1,atis_flight,what flights are available from pittsburgh to...,flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,what is the arrival time in san francisco for...,arriv time san francisco 755 flight leav wash...
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfar tacoma orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fare pittsburgh philadelphia 1000 ...


In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()

In [79]:
X = tv.fit_transform(df_train['cleaned_text'])
y = df_train.iloc[:,0]

In [80]:
from sklearn.svm import SVC
svm = SVC()

svm = svm.fit(X,y)


In [95]:
import pickle
cl = pickle.load(open('model.pkl','rb'))
tv = pickle.load(open('cv_transfomr.pkl', 'rb'))

In [96]:
message = 'i want to fly from boston at 838 am and arrive in denver at 1110 in the morning'

In [97]:
data = [message]

In [98]:
vect = tv.transform(data).toarray()
vect

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.65747418, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [99]:
cl.predict(vect).tolist()[0]

'atis_flight'

In [93]:
my_prediction = str(svm.predict(vect)[0])
my_prediction

'atis_flight'

In [94]:
vect

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.65747418, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [86]:
svm.predict(vect).tolist()[0]

'atis_flight'

In [87]:
from sklearn.model_selection import GridSearchCV

In [88]:
param_grid = {'C': [0.1, 1, 10],  
              'gamma': ['scale', 0.001, 0.0001], 
              'kernel': ['rbf','linear','poly']}

In [89]:
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

In [70]:
grid.fit(X,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.855, total=   2.3s
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.851, total=   2.4s
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


...

[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.841, total=   2.5s
[CV] C=0.1, gamma=scale, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.853, total=   2.5s
[CV] C=0.1, gamma=scale, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.857, total=   2.4s
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.898, total=   1.4s
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.909, total=   1.4s
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.885, total=   1.3s
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.888, total=   1.4s
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] .

[CV] ........ C=1, gamma=0.001, kernel=rbf, score=0.759, total=   1.5s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ........ C=1, gamma=0.001, kernel=rbf, score=0.758, total=   1.6s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ........ C=1, gamma=0.001, kernel=rbf, score=0.758, total=   1.6s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ........ C=1, gamma=0.001, kernel=rbf, score=0.758, total=   1.6s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ........ C=1, gamma=0.001, kernel=rbf, score=0.759, total=   1.5s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.971, total=   1.1s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.975, total=   1.0s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] .

[CV] ...... C=10, gamma=0.001, kernel=poly, score=0.758, total=   1.2s
[CV] C=10, gamma=0.001, kernel=poly ..................................
[CV] ...... C=10, gamma=0.001, kernel=poly, score=0.759, total=   1.3s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.759, total=   1.6s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.758, total=   1.5s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.758, total=   1.6s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.758, total=   1.6s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.759, total=   1.6s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] .

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  4.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [72]:
# print best parameter after tuning 
print(grid.best_params_) 
  

{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}


In [73]:
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
